# langfree

> Utilities to help you work with your OpenAI language model data outside LangSmith 

This library is specifically tested against OpenAI and LangSmith.  Furthermore, this library is extremely experimental and early, use with caution.

## Install

```sh
pip install langfree
```

## How to use

Fill me in please! Don't forget code examples: